<a href="https://colab.research.google.com/github/fintechsteve/modeling-volatility/blob/master/Part_03_Modeling_Framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 03: Building a Modeling Framework

### In this section you will:


*   Understand the importance of creating holdback samples
*   Explore some of the challenges of setting up a performance metric for data science models in a backtesting framework.
*   Create a generalized function that can be used to provide in- and out-of-sample performance metrics


## Import all necessary libraries

For this piece, we will need the following packages to be available to our environment:

*   Numpy and Pandas (For data manipulation)
*   DateTime (For basic date manipulation)
*   Matplotlib (For timeseries vizualization)

If the packages are not available, install the with "pip install X"

In [ ]:
import numpy as np, pandas as pd
from datetime import datetime, timedelta
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

### Read in data from previously stored returns.pkl file



In [ ]:
with open('./returns.pkl', 'rb') as f:
    returns = pickle.load(f)
    f.close()
returns.head(10)

### The importance of creating a holdback sample

When conducting data science around model building, it is important to understand the role that time plays in your modeling.

All estimation must be done using historical data. Inferences that arise from "forward looking" data can bias our understanding and the model decision making process, so it is important that we exclude them from calculations.

Unfortunately, we are not always clear about the ways in which "forward looking" can enter our modeling process and some disciplines are notoriously lax in their approach. The following "forward looking" data should be carefully controlled:

1.   Back-casting signals and model weights over the period used for estimation (in-sample). While the performance in this sample can be a useful gut check, it is dangerous to place too much weight on these insights.

2.   Data alignment and computational lag. If you run a model using data that goes through 5pm ET on a particular day, you probably can't implement a signal based on this until close of business the following day. If the data takes longer to reach your system (e.g. arrives with a 1 day processing lag) your effective lag may be longer. Performance between the date the model data is labeled and the earliest date a trade could be implemented should not be included in out-of-sample performance.

3.   Data used for macro-parameter or hyper-parameter settings. This is where most financial data scientists get themselves into trouble. If you change the parameters for an estimation window or maximum number of lags in a model using some kind of objective critereon (be it error related, e.g. RMSE, AIC or performance related, e.g. mean return, sharpe ratio, information ratio), you cannot truly count the performance over the period used to make this decision as in-sample.

   This becomes particularly problematic if you use the entire sample to reparameterize your model (e.g. because you found that the original parameterization did not work).



## Best practice for backtesting

Proper discipline around backtesting is critical and difficult to do right. At a minimum you should:

* Create a holdback period and only look at performance during this period once you are confident you have finished changing parameters on your model.
* Determine the method of evaluating your model and what you will do if your out-of-sample performance fails to meet the in-sample performance.
* Parameterize your model using a subset of the non holdback period of data. Ideally use cross-validation methods to minimize overfitting.

Ideally you should also test your model on different out of sample periods, subsamples of securities, etc to determine the robustness of your solution. However, there is an art to doing this in such a way that your testing does not influence or induce further model tweaking.

## Creating a generic model and testing framework

To help us understand our testing framework, lets create a generic model and a separate function that allows us to evaluate the performance of a model.

We can think of a model as a function that takes in a set of input data, a set of parameters and produces a set of model weights.

In the case of this example, the model weights represent be the fraction of exposure we maintain to the DXY basket (vs simply remaining hedged in our base USD currency).

Input data are the currency returns matrix. And finally parameters include:

1) Model parameters such as lookback windows for averaging, lag structure for timeseries estimators (such as maximum number of lags on AR and MA processes).
2) Hyperparameters, such as the window over which parameter estimation vs cross-validation is done, parameters for turning signals into weights, etc.
3) Modeling framework parameters, such as the extent of the holdback parameter and any framework driven constraints on the output (e.g. rescaling out of sample weights to be forced to be uniformly distributed ex post).


## A random model with a fixed seed

To test our framework, let's create a model that generates random weights regardless of the input data and parameters.

We will make the requirement that weights in-sample are uniformly distributed over [0,1] from a modeling framework perspective. However, it is easier (and better) to trust the model to do this, given our parameters.

In [ ]:
rets = returns
framework_params = {'insample_start_date': '1975-01-02',
                    'insample_end_date': '2000-01-01'}
model_params = {'seed': 1}
np.random.seed(seed=model_params['seed'])
randwts = pd.DataFrame(np.random.rand(rets['AUD'].count(),1), index=rets.index)
randwts_insample = randwts[framework_params['insample_start_date']:framework_params['insample_end_date']]
randwts_insample = randwts_insample.rank()/randwts_insample.count()
randwts[framework_params['insample_start_date']:framework_params['insample_end_date']] = randwts_insample
randwts.plot()

In [ ]:
def random_model(rets, model_params, framework_params):
  np.random.seed(seed=model_params['seed'])
  randwts = pd.DataFrame(np.random.rand(rets['AUD'].count(),1), index=rets.index)
  randwts_insample = randwts[framework_params['insample_start_date']:framework_params['insample_end_date']]
  randwts_insample = randwts_insample.rank()/randwts_insample.count()
  randwts[framework_params['insample_start_date']:framework_params['insample_end_date']] = randwts_insample
  return randwts


In [ ]:
framework_params = {'insample_start_date': '1975-01-02',
                    'insample_end_date': '2000-01-01'}
model_params = {'seed': 1}
randwts = random_model(returns, model_params, framework_params)
randwts.head(20)

In [ ]:
randwts.plot()

### Create an evaluation function that calculates the relationship between weight and subsequent DXY index volatility

The measure we are interested in is the subsequent volatility of the dxy index. For the sake of argument, we will simply use squared returns to proxy volatility (Generally currencies shouldn't have a trend if implemented using forwards to avoid interest rate differentials).

Start by calculating the dxy index again:

In [ ]:
dxy_weight = [0, 0.119, 0.036, 0, 0.136, 0.576, 0, 0, 0.091]
dxy = pd.DataFrame(returns.dot(dxy_weight))
dxy.head(10)


Now create forward looking squared returns. Note that we need to decide what horizon we measure volatility over. We assume here that it is an end of day decision to hedge volatility for the next day, so a 1 day forward lookahead rule is sufficient.

In [ ]:
dxy_forward = dxy.shift(-1)
dxy_forward_sq = dxy_forward**2
dxy_forward_sq.head(10)


We need a function that measures the volatility of a strategy by multiplying our weights by the forward-looking squared returns

In [ ]:
model_vol = randwts*dxy_forward_sq
model_vol.plot()


The problem with this is that a model biased towards zero has a lower volatility. We need a measure that takes into account the overall bias of the model and only gives credit for the ability to selectively identify periods of higher volatility. 

Note: Depending on what you are trying to measure, there is a choice to either demean over:
a) just the in sample period.
b) the whole sample.
c) the in and out of sample periods separately.

Method (b) risks introducing forward looking biases. Method (c) is appropriate if you believe that the performance out of sample will be systematically biased vs in sample. For the example below we use method (a)


In [ ]:
def model_perf(model_wts, dxyfsq, framework_params):
  model_vol = (model_wts)*dxyfsq
  avgmodel_vol = np.mean(model_wts.loc[framework_params['insample_start_date']:framework_params['insample_end_date']])*dxyfsq
  is_rmse = np.sqrt(np.mean(model_vol.loc[framework_params['insample_start_date']:framework_params['insample_end_date']]))[0]
  out_rmse = np.sqrt(np.mean(model_vol.loc[framework_params['outofsample_start_date']:framework_params['outofsample_end_date']]))[0]

  is_avg_rmse = np.sqrt(np.mean(avgmodel_vol.loc[framework_params['insample_start_date']:framework_params['insample_end_date']]))[0]
  out_avg_rmse = np.sqrt(np.mean(avgmodel_vol.loc[framework_params['outofsample_start_date']:framework_params['outofsample_end_date']]))[0]

  model_perf_stats = {'insample rmse': is_rmse, 'insample excess rmse': is_rmse - is_avg_rmse,
                      'out-of-sample rmse': out_rmse, 'out-of-sample excess rmse': out_rmse - out_avg_rmse}
  return model_perf_stats

In [ ]:
rets = returns
framework_params = {'insample_start_date': '1975-01-02',
                    'insample_end_date': '2000-01-01',
                    'outofsample_start_date': '2000-01-02',
                    'outofsample_end_date': '2017-12-26'}
model_perf_stats = model_perf(randwts, dxy_forward_sq, framework_params)
model_perf_stats

### Exploring the properties of our performance statistics with 1,000 random models

To better understand the statistics we have created, let's iterate over 1,000 random seeds and store the model performance statistics for each iteration.

In [ ]:
framework_params = {'insample_start_date': '1975-01-02',
                    'insample_end_date': '2000-01-01',
                    'outofsample_start_date': '2000-01-02',
                    'outofsample_end_date': '2017-12-26'}

def random_models_by_seed(returns, framework_params, seed):
  model_params = {'seed': seed}
  randwts = random_model(returns, model_params, framework_params)
  return model_perf(randwts, dxy_forward_sq, framework_params)

rand_perf = pd.DataFrame(list(map(lambda x:random_models_by_seed(returns, framework_params, x), range(0,1000))))

We can plot the distribution of the performance statistics. We expect that the excess RMSE should be centered around zeros - which appears to be the case.

Unsurprisingly, there is no evidence to suggest that a random model is able to predict volatility in or out of sample.

However, performing tests like this are important. They help us confirm that parts of our modeling framework are working. Given the ease of introducing coding errors, it is **vital** to do tests of the code with noise to ensure that there is no leakage of information or misinterpretation of the results from a more complex model.

In [ ]:
rand_perf.hist(bins=50)

In [ ]:
rand_perf.describe()

In [ ]:
dxy_forward_sq.to_pickle('./dxy_forward_sq.pkl')
dxy.to_pickle('./dxy.pkl')